In [1]:
import chromadb
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import datetime as dt

API_KEY = os.getenv('OAI_API_KEY')


In [2]:
df = pd.read_csv('./data/saigai_data.csv')
df = df.astype({'年': 'Int64', '月': 'Int64', '業種（中分類）': 'Int64', '業種（小分類）': 'Int64', '事故の型': 'Int64', '起因物（大分類）.2': 'Int64', '起因物（中分類）.2': 'Int64', '起因物（小分類）.2': 'Int64'})
df_new = df[['年号','年','月','災害状況','業種（大分類）.1']]

/var/folders/5s/lns070lx7x3_4_7qbhgzsj0w0000gn/T/ipykernel_15090/2698770361.py:1: DtypeWarning: Columns (7,14,16,17,18,22,24,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/saigai_data.csv')


In [3]:
client = chromadb.Client()

In [4]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=API_KEY,
                model_name="text-embedding-ada-002"
            )
collection = client.create_collection(name="disaster_collection",embedding_function=openai_ef)


In [12]:
df_dic = df_new[:1000].to_dict()
docs = list(df_dic['災害状況'].values())
# df_dicから'災害状況'キーを除いた辞書を取り出すコード
original_data = {key: value for key, value in df_dic.items() if key != '災害状況'}
metadata = [
    {key: value[index] for key, value in original_data.items()}
    for index in range(len(next(iter(original_data.values()))))
]

ids = ["id{}".format(i) for i in df_dic['災害状況'].keys()]



In [13]:
collection.add(
    documents=docs,
    metadatas=metadata,
    ids=ids
)

In [14]:
results = collection.query(
    query_texts=["季節性災害"],
    n_results=10
)
results

{'ids': [['id128',
   'id432',
   'id436',
   'id615',
   'id191',
   'id116',
   'id286',
   'id611',
   'id6',
   'id56']],
 'distances': [[0.3918856382369995,
   0.3959372937679291,
   0.39612558484077454,
   0.3994092345237732,
   0.4008343815803528,
   0.40150177478790283,
   0.40164804458618164,
   0.40415099263191223,
   0.4056999683380127,
   0.4065854847431183]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '保健・衛生業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '保健・衛生業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'}]],
 'embeddings': None,
 'documents': [['コンテナより荷卸しした製材をリフトにより倉庫内に